In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
import seaborn as sns
from rasterio.plot import show
import seaborn as sns
from sklearn.cluster import KMeans
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools.traveltime.models import rnn
from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory, busnetwork
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [ ]:
depot_ratio = 7 / 5530 # depots/km^2
deadhead_eff = 0.49 * 1.609 # km/kWh
epsg = 32148

### Energy Use for Full KCM Network

In [ ]:
network_name = "kcm"

res_dir = Path("..","results","energy", network_name)

file = open(res_dir / "trajectories.pkl", "rb")
trajectories = pickle.load(file)
file.close()

file = open(res_dir / "cycles.pkl", "rb")
cycles = pickle.load(file)
file.close()

In [ ]:
importlib.reload(busnetwork)
kcm_energy_res, kcm_depot_locations = busnetwork.get_energy_results(trajectories, cycles, n_depots=int(5530*depot_ratio), deadhead_eff=0.49)
kcm_energy_res = kcm_energy_res[kcm_energy_res['efficiency'] < 4]

print(f"Mean trip efficiency (mi/kWh): {kcm_energy_res['efficiency'].mean():2f}")
print(f"Mean trip deadhead (kWh): {kcm_energy_res['trip_deadhead_kwh'].mean():2f}")
print(f"Mean block energy (kWh): {kcm_energy_res.groupby('block_id')[['total_kwh']].sum()['total_kwh'].mean():2f}")
print(f"Mean start deadhead (kWh): {kcm_energy_res.groupby('block_id').first()['block_deadhead_start_kwh'].mean():2f}")
print(f"Mean end deadhead (kWh): {kcm_energy_res.groupby('block_id').first()['block_deadhead_end_kwh'].mean():2f}")

kcm_energy_res

In [ ]:
efficiencies = kcm_energy_res['efficiency']
fig, ax = plt.subplots()
sns.histplot(efficiencies, kde=True, ax=ax)
ax.set_title(f"BEB Trip Efficiency Distribution\n{len(efficiencies)} Routes")
ax.set_xlabel("Efficiency (mi/kWh)")
plt.show()

In [ ]:
plot_df = kcm_energy_res.groupby('block_id').first()
plot_df = gpd.GeoDataFrame(plot_df, geometry=gpd.points_from_xy([t.x for t in plot_df['start_loc']], [t.y for t in plot_df['start_loc']])).set_crs(epsg)
kcm_depot_locations = gpd.GeoDataFrame(kcm_depot_locations, geometry=gpd.points_from_xy(kcm_depot_locations['depot_x'], kcm_depot_locations['depot_y'])).set_crs(epsg)

fig, axes = plt.subplots(1,1, figsize=(10,10))
plot_df.plot(ax=axes, column='depot_id', markersize=10, cmap='tab20')
kcm_depot_locations.plot(ax=axes, marker='x', markersize=100, color="blue", linewidth=3)
cx.add_basemap(ax=axes, crs=plot_df.crs.to_string(), source=cx.providers.CartoDB.Positron)

In [ ]:
block_total_energy_kwh = kcm_energy_res.groupby('block_id').agg({'total_kwh': 'sum', 'trip_deadhead_kwh': 'sum', 'block_deadhead_start_kwh': 'first', 'block_deadhead_end_kwh': 'first'})
total_energy_kwh = block_total_energy_kwh['total_kwh'] + block_total_energy_kwh['trip_deadhead_kwh'] + block_total_energy_kwh['block_deadhead_start_kwh'] + block_total_energy_kwh['block_deadhead_end_kwh']

fig, axes = plt.subplots(1,1, figsize=(12,6))
sns.ecdfplot(total_energy_kwh, ax=axes)
axes.axvline(x=466, linestyle='dashed', color='r')
axes.text(470, 0.4, "New Flyer XE40", color='black', fontsize=12)
axes.set_title("Net Energy Consumption")
axes.set_xlabel("Vehicle Available SoC (kWh)")
axes.set_ylabel("Proportion of Blocks Met")
plt.show()

### Draw Energy Predictions for All Networks

In [ ]:
# Does a higher GTFS-RT sampling frequency improve energy estimates of the model?
# Which blocks are BEBs viable on, and what is the cost in infrastructure of electrifying them?

# How accurately can bus energy consumption be predicted using a combination of route
# geometry, elevation and models trained on standardized open bus data to inform system
# design for transit agencies transitioning to BEBs?
# 2. Given demand estimates from different sources (AVL, GTFS-RT, onboard logger) com-
# bined with a power consumption modeling framework:
# (a) Which blocks are BEBs viable on?
# (b) How do design parameters such as battery and charger sizing affect the decision
# of which blocks to electrify?
# (c) How does the availability of data sources (e.g., static only, static and realtime
# sample) impact energy predictions?

# Calculate operating parameters from GTFS + TT model
# Given GTFS energy, operating profile, design strategy, cost assumptions: Compare energy needs/electrification barriers across wide array of agencies
# Calculate drive cycle/energy use for full GTFS feed
# Growth in GTFS-RT, GTFS-Flex, API sampling resolutions, Simulate/propose change in GTFS-RT sampling frequency
# Where is GPS precision high/low in network? Energy implications?
# Energy models as constraint in blocking software
# BEB abatement curve?

In [ ]:
# Load and combine energy results from all bus networks
cleaned_sources = pd.read_csv(Path('..', 'data', 'cleaned_sources.csv'))
cleaned_sources['busnetwork_name'] = cleaned_sources['subdivision_name'] + "_" + cleaned_sources['municipality'] + "_" + cleaned_sources['provider']
all_energy_res = []
all_depot_locations = []
for i, row in cleaned_sources.iterrows():
    try:
        # Load network results
        network_id = row['uuid']
        file = open(Path("..","results","energy",network_id) / "trajectories.pkl", "rb")
        trajectories = pickle.load(file)
        file.close()
        file = open(Path("..","results","energy",network_id) / "cycles.pkl", "rb")
        cycles = pickle.load(file)
        file.close()
        # Post process
        bbox = gpd.GeoDataFrame(geometry=gpd.points_from_xy([row['min_lon'], row['max_lon']], [row['min_lat'], row['max_lat']])).set_crs(4326).to_crs(row['epsg_code'])
        bbox_area_km2 = (bbox['geometry'].iloc[1].x - bbox['geometry'].iloc[0].x) * (bbox['geometry'].iloc[1].y - bbox['geometry'].iloc[0].y) / (1000 * 1000)
        n_depots = min([max([int(bbox_area_km2 * depot_ratio), 1]), 10])
        energy_res, depot_locations = busnetwork.get_energy_results(trajectories, cycles, n_depots=n_depots, deadhead_eff=0.49)
        energy_res['network_id'] = network_id
        depot_locations['network_id'] = network_id
        # Combine
        all_energy_res.append(energy_res)
        all_depot_locations.append(depot_locations)
    except FileNotFoundError:
        print(f"Network {network_id} not found")

all_energy_res = pd.concat(all_energy_res)
all_depot_locations = pd.concat(all_depot_locations)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(18,5))
sns.boxplot(x="network_id", y="efficiency", data=all_energy_res[all_energy_res['efficiency']<4], order=all_energy_res.groupby("network_id")[['efficiency']].median().sort_values("efficiency").index, ax=axes)
plt.xticks(rotation=90)
plt.xlabel("")
plt.ylabel("Trip Efficiency (mi/kWh)")
plt.show()

In [ ]:
block_total_energy_kwh = all_energy_res.groupby(['network_id','block_id'], as_index=False).agg({'total_kwh': 'sum', 'trip_deadhead_kwh': 'sum', 'block_deadhead_start_kwh': 'first', 'block_deadhead_end_kwh': 'first'})
block_total_energy_kwh['total_energy_kwh'] = block_total_energy_kwh['total_kwh'] + block_total_energy_kwh['trip_deadhead_kwh'] + block_total_energy_kwh['block_deadhead_start_kwh'] + block_total_energy_kwh['block_deadhead_end_kwh']

fig, axes = plt.subplots(1,1, figsize=(12,6))
sns.ecdfplot(data=block_total_energy_kwh, x='total_energy_kwh', hue='network_id', ax=axes, legend=False)
axes.axvline(x=466, linestyle='dashed', color='r')
axes.text(470, 0.4, "New Flyer XE40", color='black', fontsize=12)
axes.set_xlim(0, 1000)
axes.set_title("Distribution of Block Energy Needs")
axes.set_xlabel("Vehicle Available SoC (kWh)")
axes.set_ylabel("Proportion of Blocks Met")
plt.show()

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(18,5))
plot_df = block_total_energy_kwh.groupby('network_id', as_index=False).agg({'total_kwh': 'sum'})
plot_df = pd.merge(plot_df, cleaned_sources[['uuid', 'busnetwork_name']], left_on='network_id', right_on='uuid')
plot_df_sorted = plot_df.sort_values('total_kwh', ascending=False)
sns.barplot(plot_df, y='busnetwork_name', x='total_kwh', order=plot_df_sorted['busnetwork_name'])
axes.set_title("Total Weekday Energy Needs by Bus Network")
axes.set_ylabel("")
axes.set_xlabel("Total Energy Needs (kWh)")
plt.show()

In [ ]:
plot_df['total_kwh'] / 5